In [14]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import json
import requests
import logging
import traceback

In [3]:
root_dir = '../../sdc.broadband_dev'
print(os.path.isdir(root_dir))

True


In [4]:
paths = [path for path in Path(root_dir).rglob("distribution/**/*.csv.xz")]
print(len(paths))

13


In [5]:
tdf = pd.read_csv(paths[0], low_memory=False)

In [6]:
tdf

,geoid,region_type,region_name,year,measure,value,measure_type,measure_units
0,516500115004,block group,"Block Group 4, Census Tract 115, Hampton city,...",2021,perc_income_avg_nat_package,0.723532,percent,NaN
1,517100043003,block group,"Block Group 3, Census Tract 43, Norfolk city, ...",2021,perc_income_avg_nat_package,8.805320,percent,NaN
2,511990510002,block group,"Block Group 2, Census Tract 510, York County, ...",2021,perc_income_avg_nat_package,1.209449,percent,NaN
3,510594327021,block group,"Block Group 1, Census Tract 4327.02, Fairfax C...",2021,perc_income_avg_nat_package,0.713496,percent,NaN
4,515102003032,block group,"Block Group 2, Census Tract 2003.03, Alexandri...",2021,perc_income_avg_nat_package,5.553949,percent,NaN
...,...,...,...,...,...,...,...,...
13353,24025,county,"Harford County, Maryland",2021,perc_income_avg_nat_package,0.861498,percent,NaN
13354,24011,county,"Caroline County, Maryland",2021,perc_income_avg_nat_package,1.309731,percent,NaN
13355,24015,county,"Cecil County, Maryland",2021,perc_income_avg_nat_package,0.998868,percent,NaN
13356,24009,county,"Calvert County, Maryland",2021,perc_income_avg_nat_package,0.702570,percent,NaN


In [11]:
cols = requests.get("https://raw.githubusercontent.com/uva-bi-sdad/data_repo_structure/main/col_names.json").json()
print(cols)

['measure', 'measure_type', 'region_name', 'region_type', 'value', 'year', 'moe']


# Add empty columns

In [19]:
def fix_cols(req_cols, dirpath):
    report = ""

    for dir in os.listdir(dirpath):
        subdir = os.path.join(dirpath, dir)
        if not os.path.isdir(subdir):
            continue
        report += "<h3> %s </h3>\n" % (dir)
        for path in Path(subdir).rglob("distribution/**/*"):
            logging.debug("\tEvaluating: %s" % path.name)

            if not os.path.isfile(path):
                # if path is not a file, skip to the next file to check
                continue

            parent_dir = path.parent

            if path.suffix in [".xz", ".csv"]:
                full_path = path.name
                try:
                    df = pd.read_csv(path.resolve())
                    cols = set(df.columns)
                    is_valid = len(req_cols - cols) == 0

                    if is_valid:
                        report += "\t<p>[VALID] %s</p>\n" % (full_path)
                    elif len(cols.intersection(req_cols)) > 1:
                        report += "\t<p>[MISSING %s] Needs: %s, Has: %s, %s </p>\n" % (
                            len(req_cols - cols),
                            req_cols - cols,
                            cols,
                            full_path,
                        )
                        
                        # add all the missing columns
                        for missing in (req_cols - cols):
                            df[missing] = None
                            df.to_csv(path.resolve())
                            report += 'Column [%s] added' % missing
                    else:
                        report += "\t<p>[NO OVERLAP] %s</p>\n" % (full_path)
                except:
                    print(traceback.format_exc())
                    report += "\t<p>[ERROR] %s</p>\n" % (full_path)
    return report

In [20]:
report = fix_cols(set(cols), root_dir)
print(report)

<h3> legacy </h3>
<h3> Wireless </h3>
<h3> .venv </h3>
<h3> .github </h3>
<h3> Satellite </h3>
<h3> Wired </h3>
	<p>[VALID] ncr_broadbandnow_acs_sdad_2021_perc_income_on_internet.csv.xz</p>
	<p>[VALID] ncr_broadbandnow_acs_sdad_2021_perc_income_on_internet.csv.xz</p>
	<p>[VALID] ncr_broadbandnow_acs_sdad_2021_perc_income_on_internet.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2017_2019_broadband_withoutint_compdev.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2017_2019_broadband_withoutint_compdev.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2017_2019_broadband_withoutint_compdev.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2017_2019_broadband_withoutint_compdev.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2019_2021q3_percent_above_threshold.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2019_2021q3_speed_measurements.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2019_2021q3_percent_above_threshold.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2019_2021q3_percent_above_threshold.csv.xz</p>
	<p>[VALID] ncr_hdcttrbg_2019_2021q3_percent_above_threshold.csv.xz</p>